In [1]:
from typing import Type, Tuple

import numpy as np
import matplotlib.pyplot as plt
from nnfs.datasets import spiral_data

In [2]:
np.random.seed(0)

In [54]:
class ActivationFunction:
    def __init__(self, type: str) -> None:
        self.type = type
        


class ReLu(ActivationFunction):
    def __init__(self) -> None:
        super().__init__("Relu")
        
    def output(z: np.ndarray) -> np.ndarray:
        return np.maximum(0, z)
    
    
class Sigmoid(ActivationFunction):
    def __init__(self) -> None:
        super().__init__("Sigmoid")
        
    def output(z: np.ndarray):
        return 1 / (1 + np.exp(-z))


class Softmax(ActivationFunction):
    def __init__(self) -> None:
        super().__init__("Sigmoid")
        
    def output(z: np.ndarray):
        exp_values = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_values / np.sum(exp_values, axis=1, keepdims=True)

In [55]:
class Layer:
    def __init__(self, type: str, size: Tuple, activation: Type[ActivationFunction] | None) -> None:
        self.type: str = type
        self.size: Tuple = size
        self.activation = activation
        self.output = 0
        
    def __str__(self) -> str:
        return f"Base Layer class"

    def forward(self):
        pass

class Dense(Layer):
     
    def __init__(self, size: Tuple, activation: Type[ActivationFunction]) -> None:
        super().__init__("Dense", size, activation)
        self.weights = np.random.rand(*size) * 0.01
        self.biases = np.zeros((1, size[1]))
        
        
    def __str__(self) -> str:
        return f"- {self.type} -\nWeights:\n{self.weights}\n{'-'*15}\nBias: {self.biases}"


    def forward(self, x: np.ndarray) -> None:
        self.output = self.activation.output(np.dot(x, self.weights) + self.biases)

    

class Input(Layer):
    def __init__(self, inputs: np.ndarray) -> None:
        super().__init__("Input", inputs.size, None)
        self.output = inputs

In [71]:
class Loss:
    def __init__(self, type: str) -> None:
        self.type: str = type


class CategoricalCrossEntropy(Loss):
    def __init__(self) -> None:
        super().__init__("Categorical")
        
    def forward(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        y_pred_clipped = np.clip(y_pred, 1- 1e7, 1e7)
        
        return -np.log(y_pred_clipped[range(len(y_pred)), y_true])

In [43]:
class NeuralNetwork:
    def __init__(self) -> None:
        self.layers: np.ndarray[Type[Layer]] = np.array([])
    
    def __str__(self) -> str:
        return ",\n".join([f"Layer {layer.type} (size={layer.size})" for layer in self.layers])
    
    def add(self, layer: Type[Layer]) -> None:
        self.layers = np.append(self.layers, layer)
        
    def propagate_forward(self) -> np.ndarray:
        z = self.layers[0].forward()
        for layer in self.layers[1:]:
            z = layer.activation.output(layer.forward(z))
            
        print(z)  
        return CategoricalCrossEntropy().output(z, 1)

In [74]:
X, y = spiral_data(samples=10, classes=3)

dense1 = Dense(size=(2, 16), activation=ReLu)

dense2 = Dense(size=(16, 3), activation=Softmax)

loss = CategoricalCrossEntropy()

dense1.forward(X)
dense2.forward(dense1.output)


array([1.09861229, 1.09861295, 1.09861191, 1.09861229, 1.09861229,
       1.09860081, 1.09861555, 1.09862653, 1.09861658, 1.09861229,
       1.09861229, 1.09861229, 1.09860567, 1.09860351, 1.09860413,
       1.09861229, 1.09861229, 1.09861288, 1.0985909 , 1.09857794,
       1.09861229, 1.09861353, 1.09861229, 1.09861229, 1.09861669,
       1.09862856, 1.09861896, 1.09861512, 1.09861229, 1.09861229])

In [37]:
[(ux, uy) for ux, uy in zip(X, y)]

[(array([0., 0.]), 0),
 (array([0.02643262, 0.10792125]), 0),
 (array([ 0.2165211, -0.0500133]), 0),
 (array([-0.17892653, -0.28124084]), 0),
 (array([-0.32890213, -0.29892181]), 0),
 (array([-0.47760467,  0.28378822]), 0),
 (array([0.31703328, 0.58645916]), 0),
 (array([0.02011156, 0.77751771]), 0),
 (array([ 0.37020803, -0.80812714]), 0),
 (array([-0.66834576, -0.74385075]), 0),
 (array([-0., -0.]), 1),
 (array([-0.06400931,  0.09082119]), 1),
 (array([-0.12589608,  0.18311988]), 1),
 (array([0.20543324, 0.2625039 ]), 1),
 (array([0.44281352, 0.03804014]), 1),
 (array([ 0.24805564, -0.49710198]), 1),
 (array([-0.3750798 , -0.55114389]), 1),
 (array([-0.77023843,  0.10803258]), 1),
 (array([-0.71840637,  0.52346514]), 1),
 (array([0.54768082, 0.83668735]), 1),
 (array([0., 0.]), 2),
 (array([ 0.0963875 , -0.05527322]), 2),
 (array([ 0.00333353, -0.22219722]), 2),
 (array([-0.30071328, -0.14381458]), 2),
 (array([-0.22521856,  0.38315462]), 2),
 (array([-0.13247125,  0.53953067]), 2),


In [45]:
X

array([[-0.00000000e+00,  0.00000000e+00],
       [ 1.09125150e-01, -2.09136491e-02],
       [ 2.10037764e-01, -7.25730913e-02],
       [ 5.05444776e-02, -3.29478932e-01],
       [-4.38045117e-01,  7.51487854e-02],
       [-4.62962010e-01,  3.07096324e-01],
       [-9.83888719e-02,  6.59366419e-01],
       [ 7.70341748e-01,  1.07293352e-01],
       [ 8.49465298e-01, -2.61786485e-01],
       [-4.05520018e-02, -9.99177429e-01],
       [-0.00000000e+00, -0.00000000e+00],
       [-6.54894349e-02,  8.97597512e-02],
       [-1.67272321e-01,  1.46296571e-01],
       [ 3.28584458e-01,  5.60657204e-02],
       [ 4.23537468e-01, -1.34710347e-01],
       [-1.20255205e-01, -5.42384237e-01],
       [-5.33168204e-01, -4.00220078e-01],
       [-7.77777674e-01,  4.01045595e-04],
       [-1.05959827e-01,  8.82550832e-01],
       [ 8.92457360e-01,  4.51131756e-01],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 8.63184658e-02, -6.99628579e-02],
       [-1.38498005e-01, -1.73784403e-01],
       [-2.